In [1]:
import pandas as pd
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.style.use(['seaborn-whitegrid'])
plt.rcParams['font.family'] = 'Malgun Gothic'
%config InlineBackend.figure_format = 'retina'

In [13]:
import re

In [2]:
df = pd.read_csv('./data/community.csv')

In [5]:
df.head()

,contents,date,source,title,url,vote_down,vote_up
0,"● 1910년 전라/경상의 인구 경북 1,530,564 / 경남 1,365,079...",2021-06-20,ilbe,● 가세연 프락치 강용석 및 전라도 출신 홍어새끼들이 구라치는 친일파 매국노의 고장...,https://www.ilbe.com/view/11349790749,0,0
1,옥쇄들고 나르샤한 새끼 이새끼 문재인 깜빵 안보내고 오스트리아식 개헌에 합의하...,2021-06-20,ilbe,"김무성, 유승민, 이준석 이 배신자 계파새끼들이 ㄹㅇ 나라 망친 주범이다",https://www.ilbe.com/view/11349791369,0,11
2,사실 지금까지 윤석열 주변에 김무성 측근들이 엄청 많이 침투해 있었음 (대표적으로 ...,2021-06-20,ilbe,※ 윤석열 대변인 이동훈 (전 조선일보 논설위원) 사퇴는 윤석열이 국민의힘 입당 원...,https://www.ilbe.com/view/11349731219,0,2
3,▶ 어차피 강용석 같은 부류들은 끝까지 함께 가기가 힘든 인간 유형이라는 점은 가...,2021-06-20,ilbe,▶ 가세빠가사리 연구원 / 가로세로연구소 그리고 강용석 소장 결별 할 때 된 듯 (...,https://www.ilbe.com/view/11349722991,0,0
4,1. 보수유투버들 코인팔이 대상이 탄핵무효 태극기세력들임. 2. 이준석이 보수유투...,2021-06-20,ilbe,보수유투버들이 이준석 씹는 이유,https://www.ilbe.com/view/11349708580,0,4


In [4]:
df['source'].value_counts()

mlbpark     2934
ilbe        1990
clien       1947
ppomppu     1926
dcinside    1344
inven        947
Name: source, dtype: int64

In [6]:
import MeCab

In [7]:
mecab = MeCab.Tagger()

In [8]:
one_list = list('젤펄물향색립손짱굳양톤물티밤코빛붓굿돈팩맥맛감웜집샵끈선속밖겉봄반광팁샷솔볼폼딥입펜발템갑값'
                '똥룩꿀떡존몸핏땀핫옆꽃액칠금쿨차밥병솜망폰링통옷목팔숱배살털컬홈짝뷰')

In [15]:
def parse_text(text):
    bucket = []
    start = 0
    pattern = re.compile(".*\t[A-Z]+") 
    try:
        result = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]]
    except Exception: # 데이터가 패턴이 없을때의 에러 handler
        print('err')
        return [('선물','X',True,0,0)]
    for token, pos in result:
        if start == 0:
            lspace = False
        elif text[start] == ' ':
            lspace = True
        else:
            lspace = False
        start = text.find(token, start)
        end = start + len(token)
        bucket.append((token, pos, lspace, start, end))
        start = end
    return bucket
def apply_noun_ext(df): 
    text, title = df['contents'], df['title']
    text = ' '.join([title,text])
    clean_text = re.sub('\[이준석\]', '', text)
    result = noun_ext(parse_text(clean_text))
    return result

def check_noun(cand):
    if len(re.findall('[^가-힣a-zA-Z0-9+]', cand)) > 0:
        return False
    else:
        return True
def noun_ext(parsed_text):
    global stopwords
    global one_list
    noun_bucket = []
    bi_bucket = []
    tri_bucket = []
    last_last_add = None
    last_add = None
    last_lspace = None
    for ichunk in parsed_text:
        token = ichunk[0]
        pos = ichunk[1]
        lspace = ichunk[2]
        add = False
        if pos == 'NNG': 
            if check_noun(token) == True: 
                if token not in stopwords:
                    if len(token) > 1: 
                        add = True
                    elif token in one_list:
                        add = True
        if pos == 'NNP': 
            if token not in stopwords:
                if len(token) > 1:
                    add = True
                elif token in one_list:
                    add = True
                    
        if add == True:
            if (last_add == True) and (lspace == False) and (len(noun_bucket) > 0):
                bi_bucket.append('|'.join([noun_bucket[-1], token]))
            if (last_add == True) and (last_last_add == True) and (lspace == False) and \
                                        (last_lspace == False) and (len(noun_bucket) > 1):
                tri_bucket.append('|'.join([noun_bucket[-2], noun_bucket[-1], token]))
            noun_bucket.append(token)
        last_lspace = lspace
        last_last_add = last_add
        last_add = add
    return pd.Series((noun_bucket, bi_bucket, tri_bucket))

def get_stopwords(collection):
    stopwords = []
    for i in list(collection.find({},{'_id':0,'stop':1})):
        stopwords.append(i['stop'])
    return stopwords

In [10]:
stopwords=['이준석']

In [14]:
df[['keywords','bigrams','trigrams']] = df.fillna('').apply(apply_noun_ext, axis=1)

In [16]:
df.head()

,contents,date,source,title,url,vote_down,vote_up,keywords,bigrams,trigrams
0,"● 1910년 전라/경상의 인구 경북 1,530,564 / 경남 1,365,079...",2021-06-20,ilbe,● 가세연 프락치 강용석 및 전라도 출신 홍어새끼들이 구라치는 친일파 매국노의 고장...,https://www.ilbe.com/view/11349790749,0,0,"[세연, 프락치, 강용석, 전라도, 출신, 홍어, 새끼, 구라, 친일파, 매국노, ...","[홍어|새끼, 일제|시대, 곡창|지대, 일제|시대, 궁민|구조, 고향|세탁, 홍어|...",[공업|발|전도]
1,옥쇄들고 나르샤한 새끼 이새끼 문재인 깜빵 안보내고 오스트리아식 개헌에 합의하...,2021-06-20,ilbe,"김무성, 유승민, 이준석 이 배신자 계파새끼들이 ㄹㅇ 나라 망친 주범이다",https://www.ilbe.com/view/11349791369,0,11,"[김무성, 유승민, 배신자, 계파, 새끼, 나라, 주범, 옥쇄, 새끼, 새끼, 문재...","[계파|새끼, 오스트리|아식, 우파|인척, 문재인|일당, 범죄자|새끼]",[]
2,사실 지금까지 윤석열 주변에 김무성 측근들이 엄청 많이 침투해 있었음 (대표적으로 ...,2021-06-20,ilbe,※ 윤석열 대변인 이동훈 (전 조선일보 논설위원) 사퇴는 윤석열이 국민의힘 입당 원...,https://www.ilbe.com/view/11349731219,0,2,"[윤석열, 대변인, 이동훈, 조선일보, 논설위원, 사퇴, 윤석열, 국민, 입당, 원...","[언론|방송, 거리|두기, 호남|향우회, 함정|카드, 양강|구도, 정권|심판, 천하...",[]
3,▶ 어차피 강용석 같은 부류들은 끝까지 함께 가기가 힘든 인간 유형이라는 점은 가...,2021-06-20,ilbe,▶ 가세빠가사리 연구원 / 가로세로연구소 그리고 강용석 소장 결별 할 때 된 듯 (...,https://www.ilbe.com/view/11349722991,0,0,"[가세, 빠가사리, 연구원, 가로세로, 연구소, 강용석, 소장, 결별, 국민, 경선...","[가세|빠가사리, 가로세로|연구소, 전당|대회, 호남|향우회, 가로세로|연구소, 구...",[]
4,1. 보수유투버들 코인팔이 대상이 탄핵무효 태극기세력들임. 2. 이준석이 보수유투...,2021-06-20,ilbe,보수유투버들이 이준석 씹는 이유,https://www.ilbe.com/view/11349708580,0,4,"[보수, 유투, 버들, 씹는, 이유, 보수, 유투, 버들, 코인, 팔, 대상, 탄핵...","[보수|유투, 유투|버들, 보수|유투, 유투|버들, 코인|팔, 탄핵|무효, 태극기|...","[보수|유투|버들, 보수|유투|버들, 보수|유투|버들]"


In [17]:
from collections import Counter
c = Counter()
c_b = Counter()
c_t = Counter()

for i,row in df.iterrows():
    keywords = row['keywords']
    bigrams = row['bigrams']
    trigrams = row['trigrams']
    c.update(keywords)
    c_b.update(bigrams)
    c_t.update(trigrams)
keyword = [{'word':i[0],'num':i[1]} for i in c.most_common(100)]
bigram =[{'word':i[0],'num':i[1]} for i in c_b.most_common(100)]
trigram =[{'word':i[0],'num':i[1]} for i in c_t.most_common(100)]

In [20]:
pd.DataFrame(keyword).head(20)

,word,num
0,대표,5465
1,윤석열,3284
2,민주당,3233
3,국민,2851
4,생각,2440
5,사람,2384
6,정치,2323
7,대통령,2021
8,유승민,1882
9,대선,1525


In [21]:
pd.DataFrame(bigram).head(20)

,word,num
0,여론|조사,716
1,최고|위원,579
2,반|페미,373
3,정권|교체,326
4,기능|요원,282
5,산업|기능,281
6,부정|선거,275
7,운세|지수,228
8,전당|대회,149
9,전수|조사,133


In [22]:
pd.DataFrame(trigram).head(20)

,word,num
0,산업|기능|요원,280
1,정책|센터|소장,61
2,본인|사무실|설치,58
3,대표|선거|결과,51
4,좌파|시민|단체,48
5,정보|통신|산업,46
6,병역|특례|의혹,34
7,정권|교체|가능,29
8,여성|희망|복무,15
9,남녀|동수|포럼,12


In [23]:
df[(df['source']=='ilbe') & (df['vote_up']>0)]

,contents,date,source,title,url,vote_down,vote_up,keywords,bigrams,trigrams
1,옥쇄들고 나르샤한 새끼 이새끼 문재인 깜빵 안보내고 오스트리아식 개헌에 합의하...,2021-06-20,ilbe,"김무성, 유승민, 이준석 이 배신자 계파새끼들이 ㄹㅇ 나라 망친 주범이다",https://www.ilbe.com/view/11349791369,0,11,"[김무성, 유승민, 배신자, 계파, 새끼, 나라, 주범, 옥쇄, 새끼, 새끼, 문재...","[계파|새끼, 오스트리|아식, 우파|인척, 문재인|일당, 범죄자|새끼]",[]
2,사실 지금까지 윤석열 주변에 김무성 측근들이 엄청 많이 침투해 있었음 (대표적으로 ...,2021-06-20,ilbe,※ 윤석열 대변인 이동훈 (전 조선일보 논설위원) 사퇴는 윤석열이 국민의힘 입당 원...,https://www.ilbe.com/view/11349731219,0,2,"[윤석열, 대변인, 이동훈, 조선일보, 논설위원, 사퇴, 윤석열, 국민, 입당, 원...","[언론|방송, 거리|두기, 호남|향우회, 함정|카드, 양강|구도, 정권|심판, 천하...",[]
4,1. 보수유투버들 코인팔이 대상이 탄핵무효 태극기세력들임. 2. 이준석이 보수유투...,2021-06-20,ilbe,보수유투버들이 이준석 씹는 이유,https://www.ilbe.com/view/11349708580,0,4,"[보수, 유투, 버들, 씹는, 이유, 보수, 유투, 버들, 코인, 팔, 대상, 탄핵...","[보수|유투, 유투|버들, 보수|유투, 유투|버들, 코인|팔, 탄핵|무효, 태극기|...","[보수|유투|버들, 보수|유투|버들, 보수|유투|버들]"
5,불구경 다음으로 좆되는거 구경이다ㅋ 선발 모집공고에는 재학생으로 되어있는데 뉴스1 ...,2021-06-19,ilbe,이준석 진짜 좆됨.jpg,https://www.ilbe.com/view/11349622868,0,9,"[불구경, 다음, 구경, 선발, 모집, 공고, 재학, 뉴스, 박태훈, 쉴드, 합격자...","[모집|공고, 이미지|브라우징, 브라우징|업체, 아빠|친구]",[이미지|브라우징|업체]
6,평소에 별 생각없었는데 오늘 카페에서 실제로 보니까 생각보다 잘생겼더라 근데 똑똑...,2021-06-20,ilbe,이준석 정도면 훈남이냐,https://www.ilbe.com/view/11349682246,0,4,"[정도, 평소, 생각, 카페, 생각]",[],[]
...,...,...,...,...,...,...,...,...,...,...
2205,,2021-06-01,ilbe,이준석 대표 당선을 바란다,https://www.ilbe.com/view/11346084033,0,26,"[대표, 당선]",[],[]
2209,바른미래당 출신 애들은 애초 우파였던적이 없음 바미당출신들이 뽑히는건 민주당 한번 ...,2021-06-01,ilbe,이준석이 당대표 된다고??ㅋㅋ,https://www.ilbe.com/view/11346021534,0,81,"[대표, 미래, 출신, 애초, 우파, 미당, 출신, 민주당, 마찬가지, 새끼, 내부...","[미당|출신, 내부|총질]",[]
2213,5년전 일베에서 대통령은 압도적으로 조원진이었음 국회의원 선거도 일베에서는 황교안...,2021-06-01,ilbe,이준석이 당대표 되는 이유,https://www.ilbe.com/view/11346088390,0,2,"[대표, 이유, 대통령, 압도, 조원진, 국회의원, 선거, 황교안, 전국, 싹쓸이,...",[서울|시장],[]
2217,이게 정치인의 얼굴이 아니지 절대 감정을 얼굴에 보이지 마라 이게 정치인의 ...,2021-06-01,ilbe,이준석 얼굴에 인성이 보이노,https://www.ilbe.com/view/11346090469,0,2,"[얼굴, 인성, 정치인, 얼굴, 감정, 얼굴, 정치인, 기본, 대표, 통합, 유승민...",[],[]
